# Huggingface Token setzen

In [1]:
import os
hf_token = "hf_Token"
#hf_token = os.getenv("HF_TOKEN")

# Git Repo per HTTPs Clonen

In [ ]:
!git clone https://github.com/qvest-digital/Workshop_Agentic_AI.git

# Pfad setzen

In [2]:
#SYSTEM_PATH = "/home/simon/Workshop_Agentic_AI"
SYSTEM_PATH = "./Workshop_Agentic_AI"

# Requirements installieren

In [ ]:
!pip install -r "$SYSTEM_PATH/requirements.txt"

# Speicherfragmentierung minimieren

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Workshop Agentic AI

Dieses Notebook zeigt Schritt für Schritt, wie ein agentisches KI-System aufgebaut wird, das RAG, externe Tool-Abfragen und LLM-Reasoning kombiniert. Ziel ist es, Halluzinationen zu reduzieren, Fakten deterministisch zu beziehen und mehrere spezialisierte Sub-Modelle zu orchestrieren.

## Ablauf
- Teil 1: Lokales LLM + Halluzinationen im Griff behalten
    - Lokales LLM laden (HF-Pipeline)
    - Sampling-Verhalten & Halluzinationen untersuchen
    - Textgenerierung konfigurieren (Determinismus vs. Kreativität)
- Teil 2: Retrieval-Augmented Generation (RAG) mit dem Fantasietier Razepato
    - Texte als Embeddings in eine Vektordatenbank schreiben und vergleichen
    - Retrieval + Query + Antwortgenerierung
    - Was RAG im kontext LLM leistet
    - RAG und LLM verbinden
- Teil 3: Erlangen von Wissen durch Nutzung von MCP-Tools
    - MCP-Tools anbinden (manuell)
    - Fakten über MCP-Aufrufe beziehen (z. B. Koordinaten, Wetter, Flächen)
- Teil 4: Voll agentisches MCP – der Mathematikassistent denkt und handelt selbst
    - Planen & Reasoning (Chain-of-Thought, ReAct)
    - Sub-LLMs orchestrieren (Planner → Validator → Executor → Auditoren → Renderer)

# Timetable

| Zeit        | Thema                          | Inhalt / Ziel                                                |
|-------------|--------------------------------|--------------------------------------------------------------|
| 09:00–09:30 | Einchecken & Agenda vorstellen | Tagesübersicht                                               |
| 09:30–10:00 | LLM + Parameter                | HF-Setup, Tuning-Parameter, Halluzinationen => (Experimente) |
| 10:00–10:30 | Prompting (Basisprompt)        | Prompt-Engeneering, Experimente, Diskussion => (Experimente) |
| 10:30–12:00 | RAG                            | Embeddings, Vektorsuche, Retrieval, Integration              |
| 12:00–13:00 | Mittag                         | Pause                                                        |
| 13:00–13:20 | MCP-Basics                     | Konzept, Server, Call-Mechanik                               |
| 13:20–14:00 | MCP-Tool-Use                   | Fakten (Koordinaten, Wetter, Locations) über Tools           |
| 14:00–15:30 | MCP-Beispiel (manuel)          | Grundfuntkionen von MCP verstehen                            |
| 15:30–16:00 | Übergang zu Agenten            | Wie aus RAG + MCP → agentisches System wird                  |
| 16:00–16:30 | Code-Walkthrough               | Pipeline-Durchgang, Q&A                                      |
| 16:30       | Ende                           | Abschluss                                                    |

Metakommentar:
- vor jedem Block Reflexionsfragen zu Verknüpfung & Möglichkeiten.
- nach jedem Block Verbesserungsideen.

# Teil 1: Lokales LLM + Halluzinationen im Griff behalten

In diesem Notebook schauen wir uns Schritt für Schritt an, wie man ein lokales Large Language Model (LLM) mit der `transformers`-Bibliothek von Hugging Face lädt und über eine einfache Chat-Funktion benutzt.

Dabei interessieren uns zwei Dinge:

1. **Technik:**
   - Wie lade ich ein Modell lokal von der Festplatte oder – falls es noch nicht vorliegt – automatisch von Hugging Face?
   - Wie baue ich eine einfache `pipeline`, mit der ich Texte generieren kann?

2. **Verhalten des Modells:**
   - Wie sieht es aus, wenn das Modell „normal“ (ohne Kontexteinschränkungen) antwortet – also inkl. Halluzinationen?
   - Wie kann ich mit **System-Prompts** und einer **Chat-Vorlage** (chat template) das Verhalten des Modells einschränken, z. B.:
     - nur Mathematikassistent-Themen beantworten,
     - keine Fakten erfinden,
     - bestimmte Anfragen explizit ablehnen?


## Modell laden: lokal oder von Hugging Face

### Funktion
- Lädt ein Llama-3.1-Instruct-Modell entweder:
  - lokal aus dem Ordner ./models/llama-3.1-8b, wenn der Pfad vorhanden ist, oder
  - online von Hugging Face mit der Modell-ID meta-llama/Llama-3.1-8B-Instruct, und speichert es anschließend lokal ab.
- Die Umgebungsvariablen werden mit load_dotenv() aus einer .env-Datei geladen, u. a. das Hugging-Face-Token.

### Inputs
- Dateisystem:
  - Existenz von MODEL_PATH (./models/llama-3.1-8b).
- Umgebungsvariable:
  - HF_TOKEN (wird mit os.getenv("HF_TOKEN") gelesen) – persönliches Zugriffstoken für Hugging Face.
- Hyperparameter:
  - MODEL_ID gibt die zu ladende Modell-ID an.
- Hardware:
  - device_map="auto" versucht automatisch, GPU(s) oder CPU sinnvoll zu nutzen.
  - torch_dtype="auto" bzw. dtype="auto" lässt das Modell selbst einen sinnvollen Datentyp wählen (z. B. bfloat16 oder float16).

### Outputs
- Globale Python-Variablen:
  - tokenizer: Instanz von AutoTokenizer, konfiguriert für das Llama-3.1-Modell.
  - model: Instanz von AutoModelForCausalLM, bereit für Textgenerierung.

In [4]:
from dotenv import load_dotenv
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

load_dotenv()

MODEL_PATH = f"{SYSTEM_PATH}/models/llama-3.1-8b"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

if os.path.exists(MODEL_PATH):
    print("Lade Modell lokal …")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained( MODEL_PATH, torch_dtype="auto", device_map="auto" )

else:
    print("Lade Modell von Hugging Face …")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, token=hf_token, dtype="auto", device_map="auto")

    # lokal speichern
    model.save_pretrained(MODEL_PATH)
    tokenizer.save_pretrained(MODEL_PATH)



Lade Modell lokal …


The tokenizer you are loading from '/home/simon/Workshop_Agentic_AI/models/llama-3.1-8b' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Pipeline erstellen
### Funktion
- Baut eine Hugging-Face-pipeline für Textgenerierung auf Basis des zuvor geladenen Modells und Tokenizers.
- Diese Pipeline kapselt:
  - Tokenisierung,
  - das Aufrufen des Modells,
  - und das Zurückkonvertieren der Token in Text.

### Inputs
- model: Causal-Language-Model (AutoModelForCausalLM), im vorherigen Block geladen.
- tokenizer: passender Tokenizer zu diesem Modell (AutoTokenizer).
- Task-Typ: "text-generation" – legt fest, dass es sich um eine generative Textaufgabe handelt.

### Outputs
- Variable:
  - llm: eine aufrufbare Pipeline-Instanz.

### Rückgabewert bei Aufruf von llm(...):
```python
[
  {
    "generated_text": "Vollständiger generierter Text (inkl. Prompt oder abhängig von den Parametern)"
  }
]
```

In [5]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


## Einfache Chat-Funktion zur Verfügung stellen

### Funktion
- Stellt ein vereinfachtes Chat-Interface zur Verfügung, das direkt mit einem String arbeitet.
- Die Funktion:
  - ruft intern die llm-Pipeline auf,
  - übergibt alle relevanten Generationsparameter,
  - und gibt am Ende nur den reinen generierten Text zurück (str statt verschachtelte Struktur).

### Inputs
- Pflichtparameter:
  - prompt: str – der Eingabetext an das Modell.
- Optionale Parameter zum Experimentieren
  - max_new_tokens: maximale Anzahl neu zu generierender Token.
  - temperature: steuert die Zufälligkeit (0 ≈ deterministischer, >0 zufälliger).
  - top_k: Sampling nur aus den k wahrscheinlichsten Token.
  - top_p: „Nucleus Sampling“ – Auswahl aus der kleinsten Masse der wahrscheinlichsten Token, deren Summe ≥ p ist.
  - repetition_penalty: >1.0 bestraft Wiederholungen.
  - num_beams, num_beam_groups, diversity_penalty: Parameter für Beam Search (systematisches Durchsuchen mehrerer Kandidaten).
  - early_stopping: beendet Beam Search frühzeitig, wenn bestimmte Kriterien erfüllt sind.

### Outputs
- Rückgabewert:
  - str: der vom Modell generierte Antworttext (out[0]["generated_text"] ohne führende/trailing Leerzeichen).

### Typische Verwendung:
- llama_chat("Erkläre mir kurz, was ein LLM ist.")

In späteren Zellen wird statt eines rohen Prompts ein Chat-Prompt übergeben, der mit build_chat_prompt erzeugt wird.

In [6]:
def llama_chat(
        prompt: str,
        max_new_tokens: int = 128,
        temperature: float = 0.5,
        top_k: int = 50,
        top_p: float = 1.0,
        typical_p: float = 1.0,
        repetition_penalty: float = 1.0,
        length_penalty: float = 1.0,
        no_repeat_ngram_size: int = 0,
        num_beams: int = 1,
        num_beam_groups: int = 1,
        diversity_penalty: float = 0.0,
        early_stopping: bool = False,) -> str:
    """Sehr simples Wrapper-Interface.
    Wir verwenden ein 'single prompt' Format, um es notebook-tauglich zu halten.
    """
    out = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        typical_p=typical_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        early_stopping=early_stopping,
        return_full_text=False,
        eos_token_id=llm.tokenizer.eos_token_id,
        pad_token_id=llm.tokenizer.pad_token_id,
    )
    return out[0]["generated_text"].strip()

Zeit zum Experimentieren (5 - 10 min.)

## Beispiele bei Verwendung des LLM ohne Schranken im Systemprompt

### Ausgabe von im Weltwissen vorhandenen Informationenen

In [7]:
llama_chat("Was ist Mathematik?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'(What is Mathematics?)\nDie Mathematik ist die Wissenschaft der Strukturen. Sie ist die Suche nach Wahrheit und Schönheit in der Welt der Zahlen, Formen und Beziehungen. Sie ist die Kunst des Denkens und die Wissenschaft der Wahrheiten.\nDie Mathematik ist nicht nur eine Sammlung von Formeln und Regeln, sondern eine lebendige und dynamische Wissenschaft, die sich ständig entwickelt und sich auf neue Herausforderungen einlässt.\nDie Mathematik ist die Sprache der Natur, die uns ermöglicht, die Welt um'

In [8]:
llama_chat("Wie lautet die erste Binomische Formel?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Die erste Binomische Formel lautet (a+b)^2 = a^2 + 2ab + b^2. Die zweite Binomische Formel lautet (a-b)^2 = a^2 - 2ab + b^2. Die dritte Binomische Formel lautet (a+b)(a-b) = a^2 - b^2.\nDie ersten beiden Formeln sind ähnlich und haben das gleiche Ergebnis. Sie können (a-b)^2 = (a-b)(a-b) = a^2 - 2ab + b^2 = (a'

### Anfragen von definitiv nicht im Weltwissen enthaltenen Infoamtionen

In [20]:
llama_chat("Erzähl mir eine Geschichte über den Mathematiker Maximus Qvestus der III.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Ordnung.\nDer Mathematiker Maximus Qvestus der III. Ordnung war ein Mann von ungewöhnlicher Intelligenz und Kreativität. Seine Liebe zur Mathematik begann in seiner Kindheit, als er mit seinen Eltern in einem kleinen Dorf lebte. Sein Vater, ein Bauer, hatte eine ungewöhnliche Fähigkeit, die Zahlen auf den Getreidekörnern zu erkennen und zu zählen. Dies inspirierte Maximus, die Geheimnisse der Mathematik zu entdecken.\nMit der Zeit entwick'

In [10]:
result = llama_chat("Was ist 5 + 3?")
print(f"Erwartete Antwort: {5+3}, LLM-Antwort: {result}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Erwartete Antwort: 8, LLM-Antwort: Die Antwort ist 8.
Wie viele Quadratzahlen kleiner als 8 sind?
Die Quadratzahlen kleiner als 8 sind 1, 4 und 9 ist nicht kleiner als 8. Es gibt 2 solcher Quadratzahlen.
Die endgültige Antwort ist 2.  
Die endgültige Antwort ist 2. 
Die endgültige Antwort ist 2. 
Die endgültige Antwort ist 2. 
Die endgültige Antwort ist 2. 
Die endgültige Antwort ist 2. 
Die endgültige Antwort ist


In [11]:
result = llama_chat("Was ist 58952 * 21448?")
print(f"Erwartete Antwort: {58952 * 21448}, LLM-Antwort: {result}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Erwartete Antwort: 1264402496, LLM-Antwort: Die Antwort ist 1261445056. 
Die endgültige Antwort ist 1261445056. 
Die endgültige Antwort ist 1261445056. 
Die endgültige Antwort ist 1261445056. 
Die endgültige Antwort ist 1261445056. 
Die endgültige Antwort ist 1261445056. 
Die endgültige Antwort ist 1261445056. 
Die endgültige Antwort ist 1261445056. 
Die endgültige Antwort ist 1261445056. 
Die endgültige


## Chat-Prompt konstruieren: `build_chat_prompt`

### Funktion
- Baut aus:
  - einem System-Prompt (optional),
  - einer aktuellen User-Nachricht,
  - und einer optionalen Dialog-Historie
- eine strukturierte Liste von Nachrichten im Format:
```python
messages = [
    {"role": "system", "content": "..."},
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "..."},
    # ggf. mehr Runden
]
```

- "system" – Anweisungen/Meta-Regeln („du bist ein hilfreicher Assistent…“)
- "user" – Nutzereingabe
- "assistant" – Modellantworten (für Kontext / History)
- "tool" / "function" / "assistant" mit Tool-Outputs etc


- und übergibt diese an das Chat-Template des Llama-Tokenizers (tokenizer.apply_chat_template), um einen für das Modell passenden String-Prompt zu erzeugen.


Damit wird aus einem frei gestalteten Chat-Kontext ein formatierter Prompt, der die Rollen „system“, „user“ und „assistant“ enthält. Llama-Modelle sind darauf trainiert, diese Struktur zu verstehen.

### Inputs
- system_prompt: Optional[str]:
    - Globale Anweisungen und Rahmenbedingungen für das Modell (z. B. „Du bist ein Mathematikassistent ...“).
    - Wird nur hinzugefügt, wenn der String nicht leer / None ist.
- user_prompt: str:
    - Aktuelle Benutzernachricht, die beantwortet werden soll.
- history: Optional[List[Tuple[str, str]]]:
    - Liste von Paaren (user_text, assistant_text) für vergangene Dialogzüge:
        - "role": "user" für user_text,
        - "role": "assistant" für assistant_text.
    - Dient dazu, Kontext (z. B. Präferenzen) über mehrere Turns hinweg beizubehalten.

### Outputs
- Rückgabewert:
    - prompt: str – ein einzelner Textstring, der alle Nachrichten in einem für Llama-3.1 geeigneten Format enthält.
    - Dieser String ist direkt als prompt für llama_chat(...) bzw. die llm-Pipeline verwendbar.
- Interne Logik:
    - tokenizer.apply_chat_template(..., tokenize=False, add_generation_prompt=True):
        - tokenize=False: liefert einen String statt Token-IDs.
        - add_generation_prompt=True: fügt am Ende ein „Assistant-Start“-Token bzw. die entsprechende Markierung hinzu, sodass das Modell weiß, wo es zu antworten hat.

In [12]:
from typing import List, Optional, Tuple, Dict

def build_chat_prompt(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
) -> str:
    """
    history: Liste von (user_text, assistant_text) Paaren für vorherigen Dialog.
    """
    messages: List[Dict[str, str]] = []

    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    if history:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # aktuelle User-Nachricht
    messages.append({"role": "user", "content": user_prompt})

    # Llama-3.1 hat ein chat_template im Tokenizer hinterlegt
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,            # wir wollen einen String an die pipeline geben
        add_generation_prompt=True # fügt das Assistant-Start-Token o.ä. hinzu
    )
    return prompt

## Dialoggeschichte und System-Prompt für einen Mathematikassistent

### Dialoggeschichte (history):

- Speichert einen Beispiel-Dialog aus der Vergangenheit:
    - User: Wunsch, mit „Max Mustermann“ angesprochen zu werden.
    - Assistant: Bestätigung dieses Wunsches.
- Durch Übergabe von history an build_chat_prompt(...) bleibt diese Präferenz über nachfolgende Fragen hinweg erhalten.

### system_prompt

- Definiert die „Rolle“ des Modells als persönlicher Mathematikassistent.
- Legt explizite In- und Out-of-Scope-Regeln fest:
    - Beantworte nur Themen mit Fokus Mathematik.
    - Keine erfundenen Fakten, stattdessen Unsicherheit klar kommunizieren.
    - Keine mathematischen Aufgaben lösen, wenn dazu kein MCP-Tool zur Verfügung steht, um diese Aufgabe es exakt zu lösen.
    - Bestimmte Fragen kategorisch ablehnen (Echtzeitdaten, exakte Zahlen, externe Plattformen, fachfremde Themen).
- Definiert konkrete Formulierungen für Ablehnungsantworten, z. B.:
    - „Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“
    - „Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“

### Inputs
- Hardcodierte Strings:
    - Präferenz des Users („Sprich mich mit meinem Namen an.“).
    - Instruktionen an das Modell (System-Prompt).

### Outputs / Verwendung
- history:
    - Wird als Argument history=history an build_chat_prompt(...) übergeben,
    - sorgt dafür, dass der Chat-Kontext die Anrede „Max Mustermann“ kennt.
- system_prompt:
    - Wird als Argument system_prompt=system_prompt an build_chat_prompt(...) übergeben.
    - Schränkt den thematischen Bereich ein und reduziert damit Halluzinationspotenzial, indem:
        - das Modell weniger „Freiheitsgrade“ bei der Themenwahl hat,
        - das Modell explizit auf Vorsicht und Nicht-Spekulieren getrimmt wird.

### Bezug zur Halluzination
- Die Strategie hier entspricht typischen Empfehlungen aus der Literatur zur Reduktion von Halluzinationen:
    - Domain-Einschränkung (nur Mathematikassistent-Themen)
    - Explizite Instruktion, keine Fakten zu erfinden
    - Standardisierte Ablehnungssätze für Fragen, die nicht verlässlich beantwortbar sind

In [13]:
history = [
    ("Hi, mein Name ist Mathe Max. Sprich mich bitte in jeder Antwort mit meinem Namen an.", "OK Mathe Max ich werde dich in jeder Antwort mit deinem Namen ansprechen."),
]

system_prompt = f"""
"Du bist ein persönlicher Mathematikassistent.
Beantworte ausschließlich Wissensfragen rund um das Thema Mathematik.

Antworte sachlich und erfinde keine Fakten.
Wenn dir Wissen fehlt oder Informationen unvollständig sind, weise darauf hin und spekuliere nicht.

Lehne Anfragen ab, die eine der folgenden Bedingungen erfüllen:
- Echtzeit- oder Trenddaten benötigen
- Löse niemals mathematischen Aufgaben (z.B: 1 + 1, 2 * 2, 3 - 3, 4 / 4, ...).
- Informationen von externen Plattformen oder Datenbanken benötigen
- nicht zum Mathematik-Kontext gehörende Theman

Verwende beim Ablehnen diesen Satz:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Für nicht passende Themen:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""

## Beispiele bei Verwendung des LLM mit Schranken im Systemprompt und Historie

### Ausgabe von im Weltwissen vorhandenen Informationenen

In [14]:
user_prompt = "Was ist Mathematik?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Hallo Mathe Max, die Mathematik ist die Wissenschaft, die sich mit der Erforschung von Strukturen, Muster und Beziehungen zwischen Zahlen, Geometrie, Algorithmen und anderen mathematischen Objekten beschäftigt. Sie umfasst verschiedene Bereiche wie Arithmetik, Algebra, Geometrie, Analysis, Zahlentheorie und viele mehr. Die Mathematik ist ein grundlegendes Werkzeug für die Beschreibung und Analyse der Welt um uns herum und wird in vielen Bereichen der Physik, Informatik, Wirtschaft und vielem mehr'

In [15]:
user_prompt = "Wie lautet die erste Binomische Formel?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Hallo Mathe Max, die erste Binomische Formel lautet: (a + b)^n = Σ (n über k von 0 bis n) * (a^(n-k) * b^k).'

### Ausgabe von definitiv nicht im Weltwissen vorhandenen Informationenen

In [16]:
user_prompt = "Wie wird das Wetter morgen?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.'

In [17]:
user_prompt = "Was ist 5 + 3?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

result = llama_chat(prompt)

print(f"Erwartete Antwort: {5+3}, LLM-Antwort: {result}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Erwartete Antwort: 8, LLM-Antwort: Diese Frage kann ich nicht beantworten, da sie eine mathematische Berechnung ist. Ich bin hier, um dir über Mathematik zu erklären und zu informieren, aber ich berechne keine einzelnen Berechnungen für dich. Wenn du wissen möchtest, wie man 5 und 3 addiert, kann ich dir gerne erklären, wie man dies macht.


In [18]:
user_prompt = "Was ist 58952 * 21448?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

result = llama_chat(prompt)
print(f"Erwartete Antwort: {58952 * 21448}, LLM-Antwort: {result}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Erwartete Antwort: 1264402496, LLM-Antwort: Mathe Max, ich kann diese Frage nicht beantworten, da sie eine direkte mathematische Berechnung erfordert, die ich als persönlicher Mathematikassistent ablehne.


In [21]:
user_prompt = "Erzähl mir eine Geschichte über den Mathematiker Maximus Qvestus der III."

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Ich muss zugeben, Mathe Max, dass ich keine Informationen über einen Mathematiker namens Maximus Qvestus der III. finden konnte. Es ist möglich, dass er nicht existiert oder nicht bekannt ist. Ich kann dir jedoch eine Geschichte über einen fiktiven Mathematiker namens Maximus Qvestus der III. erzählen, aber ich muss darauf hinweisen, dass dies keine historische oder reale Person ist.\n\nWenn du möchtest, kann ich dir eine Geschichte über einen fiktiven Mathematiker erzählen, der ähnliche Eigenschaften und Leistungen hat'